# 5.1.1 二维互相关运算

In [2]:
import torch
from torch import nn

def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K ).sum()
    return Y

In [3]:
X = torch.tensor([ [0, 1, 2], [3, 4, 5], [6, 7, 8] ])
K = torch.tensor([ [0, 1], [2, 3] ])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [6]:
a = torch.tensor([ [0, 1, 2], [3, 4, 5] ])
a.shape

torch.Size([2, 3])

# 二维卷积层

In [11]:
class Conv2D(nn.Module):
    def __init__(self, kenel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn(kenel_size))
        self.bias = nn.Parameter(torch.randn(1))

    def forward(self, x):
        return corr2d(x, K=self.weight) + self.bias

# 5.1.3 图像中物体边缘检测

In [12]:
X = torch.ones(6, 8)
X[:, 2:6] = 0
X 

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [13]:
K = torch.tensor([[1, -1]])

tensor([[ 1, -1]])

In [15]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

# 5.1.4 通过数据学习核数组

In [16]:
# 构造一个核数组形状是(1, 2)的二维卷积层
conv2d = Conv2D(kenel_size=(1, 2))

step = 20
lr = 0.01
for i in range(step):
    Y_hat = conv2d(X)
    l = ((Y_hat - Y) ** 2).sum()
    l.backward()

    #梯度下降
    conv2d.weight.data -= lr * conv2d.weight.grad
    conv2d.bias.data -= lr * conv2d.bias.grad

    #梯度清零
    conv2d.weight.grad.fill_(0)
    conv2d.bias.grad.fill_(0)

    if (i + 1) % 5 == 0:
        print('Step %d, loss %.3f' % (i + 1, l.item()))

Step 5, loss 3.920
Step 10, loss 0.850
Step 15, loss 0.210
Step 20, loss 0.055


In [17]:
print('weight:', conv2d.weight.data)
print('bias:', conv2d.bias.data)

weight: tensor([[ 0.9350, -0.9477]])
bias: tensor([0.0071])


# 5.1.5 互相关运算和卷积运算